In [32]:
import pandas as pd, numpy as np, json, re, pickle, keras

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import string
#import Keras libraries to build a neural network classifier
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
def read_data(file):
    """
    Take a json file location and
    read the file into a pandas data frame
    Args: full path to file
    Returns: pandas dataframe with data from file
    """
    
    data = []

    with open(file) as f:
        for line in f:
            data.append(json.loads(line))
        
    # convert to data frame
    
    return pd.DataFrame(data)

In [6]:
# read category data
# Had to complete the missing " and } in the file. Was getting error earlier.
df = read_data('categorized-comments.jsonl')

# check size, structure and categories

print('Size: ', len(df), '\n',
      'Shape: ', df.info(), '\n',
      'Categories: ', df.cat.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606474 entries, 0 to 606473
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   cat     606474 non-null  object
 1   txt     606474 non-null  object
dtypes: object(2)
memory usage: 9.3+ MB
Size:  606474 
 Shape:  None 
 Categories:  ['sports' 'science_and_technology' 'video_games']


In [7]:
df = df[:50000]

In [8]:
df.apply(lambda x: x.astype(str).str.lower())

,cat,txt
0,sports,barely better than gabbert? he was significant...
1,sports,should have drafted more wrs.\n\n- matt millen...
2,sports,[done](https://i.imgur.com/2yz90pm.jpg)
3,sports,no!! noo!!!!!
4,sports,ding dong the kaepers gone!!!!!! yes!!!! frida...
...,...,...
49995,sports,never does.
49996,sports,i think tosh is our best one left. napier is g...
49997,sports,"close, and we could really use a tight end..."
49998,sports,yeah but that didn't really end all that well ...


In [9]:
#remove punctuation
df['txt'] = df['txt'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))

In [10]:
stop = stopwords.words('english')

In [11]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
df['txt'] = df['txt'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [13]:
print(df['txt'])

0        Barely better Gabbert He significantly better ...
1                  Should drafted WRs Matt Millen probably
2                             Donehttpsiimgurcom2YZ90pmjpg
3                                                   No NOO
4             Ding dong Kaepers gone Yes Friday good start
                               ...                        
49995                                                Never
49996    I think Tosh best one left Napier gone Cristob...
49997                     Close could really use tight end
49998                           Yeah didnt really end well
49999                                      fucking kidding
Name: txt, Length: 50000, dtype: object


In [14]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['txt'] = df['txt'].apply(stem_sentences)

In [20]:
# set the features and classes

N_FEATURES = 5000
N_CLASSES = 3
N_UNITS = 2500

In [16]:
# create the feature matrix
#do tfidf
cv = CountVectorizer(analyzer='word',
                     stop_words=stop, 
                     max_features = N_FEATURES,
                     max_df = 0.5,
                     min_df = 3)

# create target and sample

X = cv.fit_transform(df['txt'])
y = df['cat']

# create train test split
# Splitting 75 25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [27]:
#A simple ANN can only take a linear array of features as input. Therefore, checking the train and test dataset accordingly.
print(str(X_train.shape))
print(str(X_test.shape))
print(str(y_train.shape))
print(str(y_test.shape))

(37500, 5000)
(12500, 5000)
(37500,)
(12500,)


In [39]:
print(y_train)

25772    science_and_technology
45791                    sports
25396    science_and_technology
43835                    sports
15782    science_and_technology
                  ...          
43723                    sports
32511                    sports
5192     science_and_technology
12172    science_and_technology
33003                    sports
Name: cat, Length: 37500, dtype: object


In [25]:
# initialize

classifier_seq = Sequential()

classifier_seq.add(Dense(units=500,activation="relu",input_shape=(N_FEATURES,)))
classifier_seq.add(Dense(units=50, activation="relu"))
classifier_seq.add(Dense(units=N_CLASSES, activation="softmax"))

# compile the Artificial Neural Network (ANN)

classifier_seq.compile(optimizer="adam", 
                       loss="categorical_crossentropy", 
                       metrics=["accuracy"])

In [33]:
#build the model architecture
def build_network():
    nn = Sequential()
    nn.add(Dense(500, activation = 'relu', input_shape = (N_FEATURES,)))
    nn.add(Dense(150, activation = 'relu'))
    nn.add(Dense(N_CLASSES, activation = 'softmax'))
    nn.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy']
    )
    return nn

In [35]:
X_train = X_train.todense

In [38]:
from keras.utils.np_utils import to_categorical
#y_train = pd.Series(y_train)
y_train = to_categorical(y_train, num_classes=N_CLASSES)

ValueError: invalid literal for int() with base 10: 'science_and_technology'

In [36]:
#train the model
nn_mod2 = KerasClassifier(build_fn = build_network, epochs = 200, batch_size = 128)
#nn_mod2.fit(X_train, y_train, validation_data = (X_val, y_val))
nn_mod2.fit(X_train, y_train)

ValueError: Failed to find data adapter that can handle input: <class 'method'>, <class 'numpy.ndarray'>

In [26]:
# fit ANN to the training set

classifier_seq.fit(X_train, y_train, batch_size=200, epochs=5, verbose = 1)

Epoch 1/5


ValueError: in user code:

    /Users/juge/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/juge/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/juge/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/juge/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/juge/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/juge/opt/anaconda3/lib/python3.8/site-pack

In [ ]:
#eval and calc matrix
classifier_seq.summary()

In [ ]:
# Checking loss and accuracy
loss, accuracy = classifier_seq.evaluate(X_test, y_test, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))

In [ ]:
# create prediction
y_pred = classifier_seq.predict_classes(X_test)

In [ ]:
# calculate model matrix -  accuracy, precision, recall, F1-score, and confusion matrix.
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))
print("Classification Report: ", classification_report(y_test,y_pred))
print("Accuracy: ", accuracy_score(y_test,y_pred))